In [3]:
import pandas as pd
import numpy as np
import sys

from kornmo import KornmoDataset
from geodata import get_farmer_elevation
import kornmo_utils as ku
from frostdataset import FrostDataset

%load_ext autoreload
%autoreload 2


In [2]:
def filter_by_years(years, data):
    return data[data['year'].isin(years)]

def get_interpolated_data(years, weather_feature):
    data = pd.DataFrame()

    print(f"Loading {weather_feature} data...")
    for year in years:
        tmp_df = pd.read_csv(f'../../kornmo-data-files/raw-data/weather-data/nn_interpolated/{weather_feature}/{weather_feature}_interpolated_{year}-03-01_to_{year}-10-01.csv')
        tmp_df.insert(0, 'year', year)
        data = pd.concat([data, tmp_df])

    # Drop columns containing 'Unnamed'
    data.drop(columns=[col for col in data.columns if 'Unnamed' in col], inplace=True)

    return_data = ku.normalize(data.filter(regex='day_.*'))
    columns_to_add = ['orgnr', 'year', 'longitude', 'latitude', 'elevation']
    for i, col in enumerate(columns_to_add):
        return_data.insert(i, col, data[col])

    print(f"Number of loaded entries: {return_data.shape[0]}")
    return return_data

def get_proximity_data(years, weather_feature):
    data = pd.DataFrame()
    print(f"Loading {weather_feature} data...")
    for year in years:
        tmp_df = pd.read_csv(f'../../kornmo-data-files/raw-data/weather-data/by_proximity/{weather_feature}/{weather_feature}_by_proximity_{year}-03-01_to_{year}-10-01.csv')
        tmp_df.drop(columns=['ws_id'], inplace=True)
        tmp_df.insert(0, 'year', year)
        data = pd.concat([data, tmp_df])

    return_data = ku.normalize(data.filter(regex='day_.*'))
    columns_to_add = ['orgnr', 'year']
    for i, col in enumerate(columns_to_add):
        return_data.insert(i, col, data[col])


    print(f"Number of loaded entries: {return_data.shape[0]}")
    return return_data

In [3]:
years = [2017, 2018, 2019, 2020]
frost = FrostDataset()
kornmo = KornmoDataset()
deliveries = kornmo.get_deliveries().pipe(ku.split_farmers_on_type)

elevation_data = get_farmer_elevation()
deliveries = deliveries.merge(elevation_data, on=['orgnr'], how='left').fillna(0)

deliveries["yield"] = ku.normalize(deliveries["levert"]/deliveries["areal"], 0, 1000)
deliveries["areal"] = ku.normalize(deliveries["areal"])
deliveries['fulldyrket'] = ku.normalize(deliveries['fulldyrket'])
deliveries['overflatedyrket'] = ku.normalize(deliveries['overflatedyrket'])
deliveries['tilskudd_dyr'] = ku.normalize(deliveries['tilskudd_dyr'])
deliveries['lat'] = ku.normalize(deliveries['lat'])
deliveries['elevation'] = ku.normalize(deliveries['elevation'])

deliveries["key"] = deliveries.orgnr.astype(str) + "/" + deliveries.year.astype(str)
deliveries = deliveries.set_index("key")
deliveries = filter_by_years(years, deliveries)

deliveries

Loading deliveries...
Number of deliveries loaded: 88624


,year,orgnr,kommunenr,gaardsnummer,bruksnummer,festenummer,fulldyrket,overflatedyrket,tilskudd_dyr,levert,areal,bygg,havre,hvete,rug_og_rughvete,lat,elevation,yield
key,,,,,,,,,,,,,,,,,,
811555762/2017,2017,811555762,1653,24,2,0,0.007241,0.0,0.000000,17067,0.017475,1.0,0.0,0.0,0.0,0.959116,0.051198,0.289271
811580082/2017,2017,811580082,1124,25,5,0,0.110575,0.0,0.335581,81204,0.064477,1.0,0.0,0.0,0.0,0.891830,0.024510,0.377693
811675792/2017,2017,811675792,709,2023,12,0,0.014180,0.0,0.000000,10902,0.020789,1.0,0.0,0.0,0.0,0.896483,0.014706,0.155743
811675792/2017,2017,811675792,709,2023,12,0,0.014180,0.0,0.000000,2335,0.004218,0.0,0.0,1.0,0.0,0.896483,0.014706,0.155667
811935662/2017,2017,811935662,125,207,1,0,0.014331,0.0,0.000000,33166,0.031636,0.0,0.0,1.0,0.0,0.907412,0.073529,0.312887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999659209/2020,2020,999659209,3812,31,64,0,0.007543,0.0,0.000000,32209,0.018078,0.0,0.0,0.0,1.0,0.898225,0.068627,0.528016
999662730/2020,2020,999662730,3802,247,1,0,0.043898,0.0,0.000000,100087,0.065080,0.0,1.0,0.0,0.0,0.902149,0.093137,0.461230
999662730/2020,2020,999662730,3802,247,1,0,0.043898,0.0,0.000000,39067,0.023802,0.0,0.0,1.0,0.0,0.902149,0.093137,0.488337


In [4]:
historical = ku.get_historical_production(kornmo, deliveries.year.unique(), 4)
historical = deliveries.merge(historical, how='left').fillna(0)
historical["key"] = historical.orgnr.astype(str) + "/" + historical.year.astype(str)
historical = historical.drop(columns=deliveries.columns)
historical = historical.drop_duplicates(subset='key')
historical = historical.set_index("key")
#historical = filter_by_years(years, historical)

historical

Loading historical grants data...
Historical data loaded for years 2013 to 2020.


,bygg_sum_0,hvete_sum_0,havre_sum_0,rug_og_rughvete_sum_0,bygg_sum_1,hvete_sum_1,havre_sum_1,rug_og_rughvete_sum_1,bygg_sum_2,hvete_sum_2,havre_sum_2,rug_og_rughvete_sum_2,bygg_sum_3,hvete_sum_3,havre_sum_3,rug_og_rughvete_sum_3
key,,,,,,,,,,,,,,,,
811555762/2017,1.3687,0.0000,0.0000,0.0,2.2050,0.0000,0.0000,0.0,2.0620,0.0000,0.0000,0.0,2.2445,0.0000,0.0000,0.0
811580082/2017,2.9200,0.0000,0.0000,0.0,4.7596,0.0000,0.0000,0.0,6.4999,0.0000,0.0000,0.0,11.7227,0.0000,0.0000,0.0
811675792/2017,0.0000,0.0000,0.0000,0.0,0.0000,2.8870,0.6015,0.0,0.0000,1.4844,0.2770,0.0,0.0000,2.2717,0.4869,0.0
811935662/2017,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,0.0,1.0643,2.5620,0.0000,0.0
812075322/2017,1.7688,0.4413,0.0000,0.0,3.3676,0.0000,0.0000,0.0,1.7060,1.4768,0.0000,0.0,2.2838,1.0750,0.0000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999640001/2020,14.6406,0.0000,4.8990,0.0,9.4914,33.1335,5.0059,0.0,8.4269,6.3109,0.0000,0.0,0.0000,49.4732,0.0000,0.0
999659209/2020,1.8102,0.0000,0.0000,0.0,0.0000,1.1576,0.0000,0.0,0.0000,0.0000,1.5396,0.0,1.3426,0.0000,0.0000,0.0
999662730/2020,0.0000,2.9122,10.4515,0.0,0.0000,4.7917,8.2873,0.0,0.0000,1.8919,5.8078,0.0,0.0000,4.4841,8.6888,0.0


In [5]:
sunlight_data = get_interpolated_data(years, 'sunlight')
daydegree5_data = get_interpolated_data(years, 'daydegree5').drop(columns=['longitude', 'latitude', 'elevation'])
ground_data = get_proximity_data(years, 'ground')
temp_and_precip_data = frost.get_as_aggregated(1).dropna().astype(float)
weather_data = temp_and_precip_data.merge(sunlight_data, how='left', on=['orgnr', 'year'])
weather_data = weather_data.merge(daydegree5_data, how='left', on=['orgnr', 'year'])
weather_data = weather_data.merge(ground_data, how='left', on=['orgnr', 'year'])

print(f"Merged {temp_and_precip_data.shape[1]} features of temp and precip data, {sunlight_data.shape[1]} features of sunlight data, {daydegree5_data.shape[1]} features of daydegree data, {ground_data.shape[1]} features of ground data to a total of {weather_data.shape[1]} features")

#weather_data = frost.get_as_aggregated(1).dropna().astype(float)

weather_data["key"] = weather_data.orgnr.astype(int).astype(str) + "/" + weather_data.year.astype(int).astype(str)
weather_data.drop(columns=["year", "orgnr"], inplace=True)
weather_data = weather_data.drop_duplicates(subset=["key"])
weather_data = weather_data.set_index("key")
weather_data

Loading sunlight data...
Number of loaded entries: 40616
Loading daydegree5 data...
Number of loaded entries: 40616
Loading ground data...
Number of loaded entries: 40616
Loading weather data...
Weather data entries loaded: 43157
Merged 859 features of temp and precip data, 219 features of sunlight data, 216 features of daydegree data, 216 features of ground data to a total of 1504 features


,growth_start_day,min_temp0,min_temp1,min_temp2,min_temp3,min_temp4,min_temp5,min_temp6,min_temp7,min_temp8,...,day_204,day_205,day_206,day_207,day_208,day_209,day_210,day_211,day_212,day_213
key,,,,,,,,,,,,,,,,,,,,,
811555762/2017,0.061905,0.491667,0.411667,0.421667,0.496667,0.486667,0.448333,0.391667,0.361667,0.406667,...,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
811580082/2017,0.038095,0.505000,0.521667,0.560000,0.543333,0.526667,0.473333,0.413333,0.446667,0.560000,...,0.285714,0.00,0.125,0.111111,0.000000,0.000000,0.000000,0.111111,0.000000,0.222222
811675792/2017,0.114286,0.485000,0.435000,0.503333,0.500000,0.450000,0.430000,0.390000,0.300000,0.490000,...,0.142857,0.25,0.125,0.111111,0.142857,0.111111,0.285714,0.111111,0.000000,0.111111
811935662/2017,0.114286,0.491667,0.465000,0.473333,0.466667,0.438333,0.406667,0.370000,0.411667,0.488333,...,0.142857,0.00,0.125,0.111111,0.285714,0.111111,0.142857,0.111111,0.000000,0.000000
812075322/2017,0.114286,0.458333,0.400000,0.440000,0.458333,0.430000,0.410000,0.398333,0.300000,0.440000,...,0.285714,0.50,0.125,0.222222,0.285714,0.222222,0.285714,0.222222,0.000000,0.222222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
869672092/2020,0.171429,0.383333,0.421667,0.428333,0.465000,0.435000,0.421667,0.361667,0.510000,0.455000,...,0.142857,0.25,0.000,0.222222,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000
995492008/2020,0.171429,0.360000,0.400000,0.430000,0.470000,0.435000,0.426667,0.368333,0.515000,0.445000,...,0.142857,0.25,0.000,0.222222,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000
914335027/2020,0.109524,0.495000,0.518333,0.506667,0.476667,0.435000,0.421667,0.448333,0.558333,0.540000,...,0.142857,0.25,0.125,0.222222,0.285714,0.222222,0.142857,0.111111,0.222222,0.111111


In [6]:
#Combine dataset

# sat_img_path = 'C:/'
# from sentinel.storage import SentinelDataset
# print("Reading sentinel_100x100_0.h5")
# ds0 = SentinelDataset(f"{sat_img_path}/sentinel_100x100_0.h5")
# print("Reading sentinel_100x100_1.h5")
# ds1 = SentinelDataset(f"{sat_img_path}/sentinel_100x100_1.h5")
# print("Combining both")
# SentinelDataset.combine_datasets([ds0, ds1], "E:/combined_compressed.h5", compression=4)

In [7]:
from kornmo.sentinel.storage import SentinelDataset
sat_img_path = 'E:/MasterThesisData/Satellite_Images'
#sat_img_path = 'C:/'
sd = SentinelDataset(f"{sat_img_path}/combined_uncompressed.h5")
train, val = sd.to_iterator().split(rand_seed='abc')

def add_historical(orgnr, year, data):
    if f"{orgnr}/{year}" in historical.index.values:
        h_data = historical.loc[f"{orgnr}/{year}"]
        return {'historical': h_data.values }
    else:
        return []
def add_weather(orgnr, year, data):
    if f"{orgnr}/{year}" not in weather_data.index:
        return []
    wd = weather_data.loc[f"{orgnr}/{year}"]
    # min_temps = [value for key, value in wd.items() if key.startswith("min_temp")]
    # mean_temps = [value for key, value in wd.items() if key.startswith("mean_temp")]
    # max_temps = [value for key, value in wd.items() if key.startswith("max_temp")]
    # total_rain = [value for key, value in wd.items() if key.startswith("total_rain")]
    # stacked = np.stack((min_temps, mean_temps, max_temps, total_rain), axis=1)

    return { 'weather': wd.values }

def add_grain_types(orgnr, year, data):
    samples = deliveries.loc[[f"{orgnr}/{year}"]]

    all_grains = []
    for _, row in samples.iterrows():
        sample = {}
        if row.bygg: sample["type"] = (1,0,0,0)
        elif row.havre: sample["type"] = (0,1,0,0)
        elif row.rug_og_rughvete: sample["type"] = (0,0,1,0)
        elif row.hvete: sample["type"] = (0,0,0,1)

        sample["areal"] = row["areal"]
        sample["lat"] = row["lat"]
        sample["elevation"] = row["elevation"]
        sample["yield"] = row["yield"]
        sample['fulldyrket'] = row['fulldyrket']
        sample['overflatedyrket'] = row['overflatedyrket']
        sample['tilskudd_dyr'] = row['tilskudd_dyr']
        all_grains.append(sample)

    return all_grains

train = train.with_data(add_historical, True)\
             .with_data(add_weather, True)\
             .with_data(add_grain_types, True)

val = val.with_data(add_historical, True)\
         .with_data(add_weather, True)\
         .with_data(add_grain_types, True)

#13597
#13582
#12876
#3400
#3395
#3182

100%|██████████| 3182/3182 [00:03<00:00, 1053.53it/s]


In [8]:
import h5py
with h5py.File('data/masks/nibio_disposed_properties_masks.h5', "r+") as out_file:
    print(out_file['masks']['811555762'].keys())

<KeysViewHDF5 ['2017', '2018', '2019']>


In [9]:
from kornmo.mask.mask_dataset import MaskDataset
from kornmo.mask.utils import add_mask_as_channel, apply_mask_to_image_series

mask_dataset_path = "data/masks/nibio_disposed_properties_masks.h5"
mask_dataset = MaskDataset(mask_dataset_path)
#print(mask_dataset.labels)

mask_iterator = mask_dataset.get_iterator()
mask_dict = {}
for orgnr, year, mask in mask_iterator:
    mask_dict[f'{orgnr}/{year}'] = mask

def apply_mask(orgnr, year, imgs):
    mask = mask_dict[f'{orgnr}/{year}']
    return apply_mask_to_image_series(mask, imgs)

train = train.filter(lambda orgnr, year, _,__: f"{orgnr}/{year}" in mask_dict)
val = val.filter(lambda orgnr, year, _,__: f"{orgnr}/{year}" in mask_dict)

print(f"train samples: {len(train)}")
print(f"val samples: {len(val)}")
#19003
#4706

train samples: 19003
val samples: 4706


In [10]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
from tensorflow.data.experimental import assert_cardinality
from kornmo.sentinel.transform import salt_n_pepper, rotate180, rotate90

stride = 10
def top_left(imgs):
    return imgs[...,:-stride, :-stride,:]
def top_right(imgs):
    return imgs[...,:-stride, stride:,:]
def bot_left(imgs):
    return imgs[...,stride:, :-stride,:]
def bot_right(imgs):
    return imgs[...,stride:, stride:,:]
def center(imgs):
    s = stride//2
    return imgs[...,s:-s, s:-s,:]

def rotate_random(imgs):
    angle = np.random.rand(30) * 6.28
    return tfa.image.rotate(imgs, angle)

augmented_dataset = train\
    .transform(apply_mask)\
    .transform(salt_n_pepper())\
    .augment([center, top_left, top_right, bot_left, bot_right], keep_original=False)\
    .transform(rotate_random)

def apply_output(orgnr, year, img_source, data):
    features = data["areal"], *data["type"]
    output = data["yield"]
    weather = data["weather"][1:]
    return {"cnn_input": img_source[0:30], "feature_input": features, "weather_input": weather}, output

train_dataset = tf.data.Dataset.from_generator(
    augmented_dataset.apply(apply_output).shuffled(),
    output_types=({"cnn_input": tf.dtypes.float64, "feature_input": tf.dtypes.float64, "weather_input": tf.dtypes.float64}, tf.dtypes.float64),
).apply(assert_cardinality(len(augmented_dataset)))

val_dataset = tf.data.Dataset.from_generator(
    val.transform(apply_mask).transform(center).apply(apply_output),
    output_types=({"cnn_input": tf.dtypes.float64, "feature_input": tf.dtypes.float64, "weather_input": tf.dtypes.float64}, tf.dtypes.float64),
).apply(assert_cardinality(len(val)))

print(f"Augmented samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
#95015
#4706

Augmented samples: 95015
Validation samples: 4706


In [11]:
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.models import Sequential, load_model
from tensorflow import keras



def CNN(input_dim, output_dim):
    input_layer = layers.Input(shape=input_dim)
    y = layers.Conv2D(16, (3, 3), activation=tf.nn.relu, padding='same')(input_layer)
    y = layers.MaxPool2D((2, 2))(y)
    y = layers.Conv2D(32, (3, 3), activation=tf.nn.relu, padding='same')(y)
    y = layers.MaxPool2D((2, 2))(y)
    y = layers.Conv2D(64, (3, 3), activation=tf.nn.relu, padding='same')(y)
    y = layers.MaxPool2D((2, 2))(y)
    y = layers.Flatten()(y)
    y = layers.Dense(output_dim, activation=tf.nn.relu)(y)

    return models.Model(inputs=[input_layer], outputs=[y], name="SingleImageCNN")

model_checkpoint = keras.callbacks.ModelCheckpoint(
    './training',
    monitor="val_loss",
    verbose=0,
    mode="min",
    save_best_only=True,
    save_weights_only=True,
)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

callbacks = [callback, model_checkpoint]

restart = True
if restart:

    scnn = CNN((90, 90, 12), 64)
    #scnn.summary(line_length=130)
    input_weather = layers.Input(shape=1501, name="weather_input") #shape = 856 / 1501
    t_wm = layers.Reshape((19, 79))(input_weather) # (4, 214) / (19, 79)
    t_wm = layers.Permute((2, 1))(t_wm)
    t_wm = layers.Conv1D(64, 50, activation=tf.nn.relu)(t_wm) # (64, 7, 7) / (64, 50)

    input_cnn = layers.Input(shape=(30, 90, 90, 12), name="cnn_input")

    feature_input = layers.Input(shape=(5,), name="feature_input")
    feature_repeated = layers.RepeatVector(30)(feature_input)

    cnn = layers.TimeDistributed(scnn)(input_cnn)
    cnn = layers.Concatenate(axis=2)([cnn, feature_repeated, t_wm])
    cnn = layers.GRU(128, return_sequences=False)(cnn)
    cnn = layers.Flatten()(cnn)
    cnn = layers.Dense(128, activation=tf.nn.relu)(cnn)
    cnn = layers.Dense(1)(cnn)

    cnn = models.Model(inputs=[input_weather, input_cnn, feature_input], outputs=cnn, name="CNN")
    #cnn.summary(line_length=130)

    cnn.compile(optimizer=optimizers.Adam(), loss='mean_absolute_error')

    cnn_history = cnn.fit(
        train_dataset.take(10000).batch(32).prefetch(2),
        validation_data=val_dataset.batch(32).prefetch(2),
        epochs=10,
        verbose=0,
        callbacks=callbacks
    )

else:
    cnn = load_model('./training/epoch_2.hdf5')
    # update the learning rate
    cnn_history = cnn.fit(
        train_dataset.take(10000).batch(32).prefetch(2),
        validation_data=val_dataset.batch(32).prefetch(2),
        epochs=10,
        verbose=1,
        callbacks=callbacks
    )

0epoch [00:00, ?epoch/s]
  0%|          | 0/10 [00:00<?, ?epoch/s]
  0%|          | 0.00/313 [00:00<?, ?batch/s]
  0%|          | 1.00/313 [00:14<1:13:07, 14.1s/batch, loss=0.196]
  1%|          | 2.00/313 [00:14<30:50, 5.95s/batch, loss=0.471]  
  1%|          | 3.00/313 [00:18<26:10, 5.06s/batch, loss=0.436]
  1%|▏         | 4.00/313 [00:22<24:04, 4.67s/batch, loss=0.469]
  2%|▏         | 5.00/313 [00:26<23:09, 4.51s/batch, loss=0.425]
  2%|▏         | 6.00/313 [00:31<23:30, 4.60s/batch, loss=0.4]  
  2%|▏         | 7.00/313 [00:35<23:11, 4.55s/batch, loss=0.385]
  3%|▎         | 8.00/313 [00:40<22:54, 4.50s/batch, loss=0.355]
  3%|▎         | 9.00/313 [00:44<22:30, 4.44s/batch, loss=0.342]
  3%|▎         | 10.0/313 [00:48<22:06, 4.38s/batch, loss=0.326]
  4%|▎         | 11.0/313 [00:53<21:47, 4.33s/batch, loss=0.312]
  4%|▍         | 12.0/313 [00:57<21:40, 4.32s/batch, loss=0.297]
  4%|▍         | 13.0/313 [01:01<21:34, 4.32s/batch, loss=0.288]
  4%|▍         | 14.0/313 [01:05<21:31

KeyboardInterrupt: 

In [ ]:
history = pd.read_json('training/hybrid_more_features.json')
history

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('whitegrid')
sns.set_context("paper")

plt.xlabel('Epoch')
plt.ylabel("Loss")
plt.plot(history['loss'].tolist(), label="loss")
plt.plot(history['val_loss'].tolist(), label="val_loss")
plt.legend()
plt.title("Mean absolute error loss")
plt.savefig('logs/hybrid_more_features.svg', dpi=600)
plt.grid()